# DEC Agglomerative Clustering

This notebook focuses on using Agglomerative Clustering instead of other types of clustering algorithms. The notebook will be extended to other algorithms like: Spectral Clustering, DBSCAN, refer to the webpage: https://scikit-learn.org/stable/modules/clustering.html

### Importing required Libaries

Importing the required libraries and modules so that they can be used in the notebook

In [1]:
import tensorflow as tf
import keras.backend as K
import numpy as np
import pandas as pd
# loading the requirements for the Xception model
from keras.applications.xception import Xception
from keras.applications.xception import absolute_import, decode_predictions, preprocess_input
from keras.layers import Flatten, Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
# Loading the DEC module cloned from github
from DEC.model import *
from DEC.metrics import *
from xception_dec_datagenerator import XceptionDataGenerator
# Importing the utilities
from utils.file_utils import *
from PIL import Image
# For visualization of images and for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
# PCA
from sklearn.decomposition import PCA

# Using scikit-image  resize function for resizing the image from original size to 224 X 224
# from skimage.transform import resize
# Train Test split from sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# Copy function
from shutil import copy2

Using TensorFlow backend.


ImportError: cannot import name '_validate_lengths'

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Xception Model

First we load the Xception model into the computer memory using the Keras library. Because we are focusing on extracting features from the model we do not include the topmost layer. However we do use the imagenet weights for the model. Also because we want a 1-D vector form of the features we do use the pooling layer at the end.

In [3]:
input_tensor_shape = (150, 150, 3)
base_xception_model = Xception(weights = 'imagenet', input_shape = input_tensor_shape, include_top = False, pooling='avg')
base_xception_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 74, 74, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 74, 74, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

The input shape of our base xception model is: 150 X 150 X 3. That is a 3 channel square image with side 150 pixels.
The output shape of the base xception model is: 2048 X 1. It is a 1-D vector representing the features learned by the model

### Data Generators

Defining the keras data generators to iterate through all the images and then essentially help in extracting the features from the images

In [4]:
image_extension = '.jpg'
training_directory_path = f'../data/xception_clustering/training/'
testing_directory_path = f'../data/xception_clustering/testing/'

In [5]:
# Getting the files already in the training and testing folders respectively
spiral_training_directory_path = construct_path(training_directory_path, 'spiral')
elliptical_training_directory_path = construct_path(training_directory_path, 'elliptical')
spiral_testing_directory_path = construct_path(testing_directory_path, 'spiral')
elliptical_testing_directory_path = construct_path(testing_directory_path, 'elliptical')
elliptical_training_files = get_file_nms(elliptical_training_directory_path, image_extension)
spiral_training_files = get_file_nms(spiral_training_directory_path, image_extension)
elliptical_testing_files = get_file_nms(elliptical_testing_directory_path, image_extension)
spiral_testing_files = get_file_nms(spiral_testing_directory_path, image_extension)
# Finding the number of images for each type of galaxy after finding the common images and list
print(f'Number of already present Training Elliptical Galaxies: {len(elliptical_training_files)}')
print(f'Number of already present Training Spiral Galaxies: {len(spiral_training_files)}')
print(f'Number of already present Testing Elliptical Galaxies: {len(elliptical_testing_files)}')
print(f'Number of already present Testing Spiral Galaxies: {len(spiral_testing_files)}')

Number of already present Training Elliptical Galaxies: 25868
Number of already present Training Spiral Galaxies: 34105
Number of already present Testing Elliptical Galaxies: 0
Number of already present Testing Spiral Galaxies: 0


#### Normalization and Cropping functions

In [6]:
def get_difference(orig_size, target_size):
    orig_size, target_size = list(orig_size), list(target_size)
    ret_ls = []
    for o, t in zip(orig_size, target_size):
        ret_ls.append(o - t)
    return ret_ls

def crop_image(image, orig_size, target_size):
    crop_sizes = get_difference(orig_size, target_size)
    height_dif, width_dif = crop_sizes[0] // 2, crop_sizes[1] // 2
    return image[height_dif:(height_dif + target_size[0]), width_dif:(width_dif + target_size[1]), :]

def range_scaling(image, out_feature_range=(-1, 1)):
    old_min, old_max = 0., 255.
    new_min, new_max = -1., 1.
    return ((image - old_min)/(old_max - old_min))*(new_max - new_min) + new_min

def image_preprocessing_function(image, crop=True, range_scale=True):
    """
    image is a 3-D image tensor (numpy array).
    """
    target_image_size = input_tensor_shape
    if crop:
        cropped_image = crop_image(image, image.shape, target_image_size)
    else:
        cropped_image = image
        
    if range_scale:
        final_image = range_scaling(cropped_image)
    else:
        final_image = cropped_image
    return final_image

#### Generator Definitions

In [7]:
%%time
generator_batch_size = 64
# Current generator uses -1 to 1
image_generator = ImageDataGenerator(preprocessing_function=image_preprocessing_function)
training_generator = image_generator.flow_from_directory(training_directory_path, target_size = input_tensor_shape[:2], 
                                                         class_mode='binary', batch_size=generator_batch_size)
testing_generator = image_generator.flow_from_directory(testing_directory_path, target_size = input_tensor_shape[:2], 
                                                         class_mode='binary', batch_size=generator_batch_size)

Found 59973 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
CPU times: user 3.27 s, sys: 528 ms, total: 3.8 s
Wall time: 2.88 s


In [8]:
%%time
n_train_examples = (len(training_generator.filenames)//generator_batch_size) * generator_batch_size
train_features = np.zeros((n_train_examples, 2048))
train_labels = np.zeros(n_train_examples, dtype=int)
i = 0
for inputs_batch, labels_batch in training_generator:
    features_batch = base_xception_model.predict(inputs_batch)
    train_features[i * generator_batch_size : (i + 1) * generator_batch_size] = features_batch
    train_labels[i * generator_batch_size : (i + 1) * generator_batch_size] = labels_batch
    i += 1
    if i % 100 == 0 and i:
        print('Number of Images processed:', i * generator_batch_size)
    if i * generator_batch_size >= n_train_examples:
        break

print('Shape of the training features', train_features.shape)

Number of Images processed: 6400
Number of Images processed: 12800
Number of Images processed: 19200
Number of Images processed: 25600
Number of Images processed: 32000
Number of Images processed: 38400
Number of Images processed: 44800
Number of Images processed: 51200
Number of Images processed: 57600
Shape of the training features (59968, 2048)
CPU times: user 6min 31s, sys: 1min 31s, total: 8min 2s
Wall time: 7min 39s


In [9]:
training_generator.class_indices

{'elliptical': 0, 'spiral': 1}

## Single-Link Agglomerative Clustering

### DEC Xception Training Regime

This part of the notebook defines the generator for the training regime of the DEC model over the features extracted from the Xception architecture

In [10]:
# Defining our DEC model
dec_model = DEC_Agglomerative([2048, 500, 500, 2000, 10], n_clusters=2)
dec_model.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 2048)              0         
_________________________________________________________________
encoder_0 (Dense)            (None, 500)               1024500   
_________________________________________________________________
encoder_1 (Dense)            (None, 500)               250500    
_________________________________________________________________
encoder_2 (Dense)            (None, 2000)              1002000   
_________________________________________________________________
encoder_3 (Dense)            (None, 10)                20010     
_________________________________________________________________
clustering (ClusteringLayer) (None, 2)                 20        
Total params: 2,297,030
Trainable params: 2,297,030
Non-trainable params: 0
_________________________________________________________________


In [11]:
results_save_dir = 'results/single_link_agglomerative_clustering'
if not exist_directory(results_save_dir):
    os.makedirs(results_save_dir)

#### Pretraining

In [12]:
%%time
dec_model.pretrain(train_features, train_labels, epochs=100, save_dir=results_save_dir)

...Pretraining...
Epoch 1/100
59968/59968 [==============================] - 5s 82us/step - loss: 0.0151
        |==>  acc: 0.5670,  nmi: 0.0622  <==|
Epoch 2/100
59968/59968 [==============================] - 4s 67us/step - loss: 0.0081
Epoch 3/100
59968/59968 [==============================] - 4s 66us/step - loss: 0.0071
Epoch 4/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0067
Epoch 5/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0066
Epoch 6/100
59968/59968 [==============================] - 4s 66us/step - loss: 0.0064
Epoch 7/100
59968/59968 [==============================] - 4s 66us/step - loss: 0.0063
Epoch 8/100
59968/59968 [==============================] - 4s 69us/step - loss: 0.0062
Epoch 9/100
59968/59968 [==============================] - 4s 70us/step - loss: 0.0061
Epoch 10/100
59968/59968 [==============================] - 4s 68us/step - loss: 0.0061
Epoch 11/100
59968/59968 [==============================] - 4s 71

#### Clustering

In [13]:
%%time
dec_model.compile(optimizer=SGD(0.01, 0.9), loss='kld')
y_pred = dec_model.fit(train_features, y=train_labels, batch_size=32, save_dir=results_save_dir, 
                       cluster_linkage='single')
print('acc:', metrics.acc(train_labels, y_pred))

Update interval 140
Save interval 9370
Initializing cluster centers with k-means.
Iter 0: acc = 0.56849, nmi = 0.00075, ari = -0.00006  ; loss= 0
saving model to: results/single_link_agglomerative_clustering/DEC_model_0.h5
Iter 140: acc = 0.56328, nmi = 0.00476, ari = -0.00214  ; loss= 0.00215
Iter 280: acc = 0.54769, nmi = 0.00539, ari = -0.00611  ; loss= 0.00428
Iter 420: acc = 0.53095, nmi = 0.00630, ari = -0.00803  ; loss= 0.00817
Iter 560: acc = 0.51863, nmi = 0.00616, ari = -0.00773  ; loss= 0.02923
Iter 700: acc = 0.51107, nmi = 0.00679, ari = -0.00734  ; loss= 0.02866
Iter 840: acc = 0.50664, nmi = 0.00784, ari = -0.00726  ; loss= 0.02808
Iter 980: acc = 0.50255, nmi = 0.00832, ari = -0.00683  ; loss= 0.04376
Iter 1120: acc = 0.50462, nmi = 0.00870, ari = -0.00560  ; loss= 0.03115
Iter 1260: acc = 0.51242, nmi = 0.00991, ari = -0.00419  ; loss= 0.04727
Iter 1400: acc = 0.51469, nmi = 0.00937, ari = -0.00335  ; loss= 0.02691
Iter 1540: acc = 0.51743, nmi = 0.00871, ari = -0.0022

### Visualization of the features extracted by the DEC model

#### Encoder Output Features

In [14]:
%%time
dec_encoder_model_pred = dec_model.encoder.predict(train_features)
pca_mod = PCA(2)
pca_mod.fit(dec_encoder_model_pred)
reduced_features = pca_mod.transform(dec_encoder_model_pred)

fig, ax = plt.subplots(1, 1)
sns.scatterplot(reduced_features[:, 0], reduced_features[:, 1], 
                hue=np.where(train_labels==0, 'elliptical', 'spiral'), ax=ax)
ax.text(0.02, 0.92, f'Explained Var: {np.round(np.sum(pca_mod.explained_variance_ratio_), decimals=4)}', 
        transform=ax.transAxes)
ax.set_xlabel('PCA Dim 1')
ax.set_ylabel('PCA Dim 2')
ax.set_title('PCA Scatterplot for Encoder Features')
ax.legend(loc=4)
plt.savefig(f'{results_save_dir}/PCAencoding_pca_features.png')

NameError: name 'PCA' is not defined

#### Training Curves

In [15]:
dec_log, ptrain_log = pd.read_csv(f'{results_save_dir}/dec_log.csv'), pd.read_csv(f'{results_save_dir}/pretrain_log.csv')

In [16]:
fig, ax = plt.subplots(1, 3, figsize=(20, 5))
ptrain_log.plot(x='epoch', y='loss', ax=ax[0], title='Reconstruction Loss')
dec_log.plot(x='iter', y='acc', ax=ax[1], title='Clustering Accuracy')
dec_log.plot(x='iter', y='nmi', ax=ax[-1], title='Clustering NMI')
plt.savefig(f'{results_save_dir}/training_curves.png')

NameError: name 'plt' is not defined

## Complete-Link Agglomerative Clustering

### DEC Xception Training Regime

This part of the notebook defines the generator for the training regime of the DEC model over the features extracted from the Xception architecture

In [17]:
# Defining our DEC model
dec_model = DEC_Agglomerative([2048, 500, 500, 2000, 10], n_clusters=2)
dec_model.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 2048)              0         
_________________________________________________________________
encoder_0 (Dense)            (None, 500)               1024500   
_________________________________________________________________
encoder_1 (Dense)            (None, 500)               250500    
_________________________________________________________________
encoder_2 (Dense)            (None, 2000)              1002000   
_________________________________________________________________
encoder_3 (Dense)            (None, 10)                20010     
_________________________________________________________________
clustering (ClusteringLayer) (None, 2)                 20        
Total params: 2,297,030
Trainable params: 2,297,030
Non-trainable params: 0
_________________________________________________________________


In [18]:
results_save_dir = 'results/complete_link_agglomerative_clustering'
if not exist_directory(results_save_dir):
    os.makedirs(results_save_dir)

#### Pretraining

In [19]:
%%time
dec_model.pretrain(train_features, train_labels, epochs=100, save_dir=results_save_dir)

...Pretraining...
Epoch 1/100
59968/59968 [==============================] - 5s 82us/step - loss: 0.0156
        |==>  acc: 0.6630,  nmi: 0.0757  <==|
Epoch 2/100
59968/59968 [==============================] - 4s 66us/step - loss: 0.0081
Epoch 3/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0071
Epoch 4/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0067
Epoch 5/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0065
Epoch 6/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0064
Epoch 7/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0063
Epoch 8/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0062
Epoch 9/100
59968/59968 [==============================] - 4s 64us/step - loss: 0.0061
Epoch 10/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0060
Epoch 11/100
59968/59968 [==============================] - 4s 66

#### Clustering

In [20]:
%%time
dec_model.compile(optimizer=SGD(0.01, 0.9), loss='kld')
y_pred = dec_model.fit(train_features, y=train_labels, batch_size=32, save_dir=results_save_dir, 
                       cluster_linkage='complete')
print('acc:', metrics.acc(train_labels, y_pred))

Update interval 140
Save interval 9370
Initializing cluster centers with k-means.
Iter 0: acc = 0.56844, nmi = 0.00197, ari = -0.00009  ; loss= 0
saving model to: results/complete_link_agglomerative_clustering/DEC_model_0.h5
Iter 140: acc = 0.56527, nmi = 0.00269, ari = -0.00131  ; loss= 0.00241
Iter 280: acc = 0.55636, nmi = 0.00153, ari = -0.00313  ; loss= 0.00419
Iter 420: acc = 0.54556, nmi = 0.00143, ari = -0.00426  ; loss= 0.00819
Iter 560: acc = 0.53962, nmi = 0.00088, ari = -0.00372  ; loss= 0.03223
Iter 700: acc = 0.53467, nmi = 0.00088, ari = -0.00373  ; loss= 0.0176
Iter 840: acc = 0.53037, nmi = 0.00092, ari = -0.00371  ; loss= 0.02774
Iter 980: acc = 0.52745, nmi = 0.00072, ari = -0.00321  ; loss= 0.04192
Iter 1120: acc = 0.52396, nmi = 0.00070, ari = -0.00299  ; loss= 0.03646
Iter 1260: acc = 0.51939, nmi = 0.00113, ari = -0.00349  ; loss= 0.04609
Iter 1400: acc = 0.51733, nmi = 0.00106, ari = -0.00321  ; loss= 0.0272
Iter 1540: acc = 0.51679, nmi = 0.00095, ari = -0.0029

### Visualization of the features extracted by the DEC model

#### Encoder Output Features

In [21]:
%%time
dec_encoder_model_pred = dec_model.encoder.predict(train_features)
pca_mod = PCA(2)
pca_mod.fit(dec_encoder_model_pred)
reduced_features = pca_mod.transform(dec_encoder_model_pred)

fig, ax = plt.subplots(1, 1)
sns.scatterplot(reduced_features[:, 0], reduced_features[:, 1], 
                hue=np.where(train_labels==0, 'elliptical', 'spiral'), ax=ax)
ax.text(0.02, 0.92, f'Explained Var: {np.round(np.sum(pca_mod.explained_variance_ratio_), decimals=4)}', 
        transform=ax.transAxes)
ax.set_xlabel('PCA Dim 1')
ax.set_ylabel('PCA Dim 2')
ax.set_title('PCA Scatterplot for Encoder Features')
ax.legend(loc=4)
plt.savefig(f'{results_save_dir}/PCAencoding_pca_features.png')

NameError: name 'PCA' is not defined

#### Training Curves

In [22]:
dec_log, ptrain_log = pd.read_csv(f'{results_save_dir}/dec_log.csv'), pd.read_csv(f'{results_save_dir}/pretrain_log.csv')

In [23]:
fig, ax = plt.subplots(1, 3, figsize=(20, 5))
ptrain_log.plot(x='epoch', y='loss', ax=ax[0], title='Reconstruction Loss')
dec_log.plot(x='iter', y='acc', ax=ax[1], title='Clustering Accuracy')
dec_log.plot(x='iter', y='nmi', ax=ax[-1], title='Clustering NMI')
plt.savefig(f'{results_save_dir}/training_curves.png')

NameError: name 'plt' is not defined

## Average-Link Agglomerative Clustering

### DEC Xception Training Regime

This part of the notebook defines the generator for the training regime of the DEC model over the features extracted from the Xception architecture

In [24]:
# Defining our DEC model
dec_model = DEC_Agglomerative([2048, 500, 500, 2000, 10], n_clusters=2)
dec_model.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 2048)              0         
_________________________________________________________________
encoder_0 (Dense)            (None, 500)               1024500   
_________________________________________________________________
encoder_1 (Dense)            (None, 500)               250500    
_________________________________________________________________
encoder_2 (Dense)            (None, 2000)              1002000   
_________________________________________________________________
encoder_3 (Dense)            (None, 10)                20010     
_________________________________________________________________
clustering (ClusteringLayer) (None, 2)                 20        
Total params: 2,297,030
Trainable params: 2,297,030
Non-trainable params: 0
_________________________________________________________________


In [25]:
results_save_dir = 'results/average_link_agglomerative_clustering'
if not exist_directory(results_save_dir):
    os.makedirs(results_save_dir)

#### Pretraining

In [26]:
%%time
dec_model.pretrain(train_features, train_labels, epochs=100, save_dir=results_save_dir)

...Pretraining...
Epoch 1/100
59968/59968 [==============================] - 5s 83us/step - loss: 0.0149
        |==>  acc: 0.6642,  nmi: 0.0724  <==|
Epoch 2/100
59968/59968 [==============================] - 4s 67us/step - loss: 0.0079
Epoch 3/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0070
Epoch 4/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0067
Epoch 5/100
59968/59968 [==============================] - 4s 66us/step - loss: 0.0065
Epoch 6/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0064
Epoch 7/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0063
Epoch 8/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0062
Epoch 9/100
59968/59968 [==============================] - 4s 66us/step - loss: 0.0061
Epoch 10/100
59968/59968 [==============================] - 4s 66us/step - loss: 0.0060
Epoch 11/100
59968/59968 [==============================] - 4s 66

#### Clustering

In [27]:
%%time
dec_model.compile(optimizer=SGD(0.01, 0.9), loss='kld')
y_pred = dec_model.fit(train_features, y=train_labels, batch_size=32, save_dir=results_save_dir, 
                       cluster_linkage='average')
print('acc:', metrics.acc(train_labels, y_pred))

Update interval 140
Save interval 9370
Initializing cluster centers with k-means.
Iter 0: acc = 0.56735, nmi = 0.00490, ari = -0.00059  ; loss= 0
saving model to: results/average_link_agglomerative_clustering/DEC_model_0.h5
Iter 140: acc = 0.55823, nmi = 0.00746, ari = -0.00398  ; loss= 0.00264
Iter 280: acc = 0.54219, nmi = 0.00513, ari = -0.00679  ; loss= 0.00489
Iter 420: acc = 0.52862, nmi = 0.00507, ari = -0.00750  ; loss= 0.00989
Iter 560: acc = 0.51809, nmi = 0.00506, ari = -0.00709  ; loss= 0.03567
Iter 700: acc = 0.51259, nmi = 0.00518, ari = -0.00663  ; loss= 0.023
Iter 840: acc = 0.50929, nmi = 0.00546, ari = -0.00639  ; loss= 0.02766
Iter 980: acc = 0.50439, nmi = 0.00533, ari = -0.00559  ; loss= 0.05472
Iter 1120: acc = 0.50190, nmi = 0.00490, ari = -0.00489  ; loss= 0.03293
Iter 1260: acc = 0.50419, nmi = 0.00534, ari = -0.00398  ; loss= 0.05221
Iter 1400: acc = 0.50734, nmi = 0.00510, ari = -0.00314  ; loss= 0.03244
Iter 1540: acc = 0.50754, nmi = 0.00397, ari = -0.00238

### Visualization of the features extracted by the DEC model

#### Encoder Output Features

In [28]:
%%time
dec_encoder_model_pred = dec_model.encoder.predict(train_features)
pca_mod = PCA(2)
pca_mod.fit(dec_encoder_model_pred)
reduced_features = pca_mod.transform(dec_encoder_model_pred)

fig, ax = plt.subplots(1, 1)
sns.scatterplot(reduced_features[:, 0], reduced_features[:, 1], 
                hue=np.where(train_labels==0, 'elliptical', 'spiral'), ax=ax)
ax.text(0.02, 0.92, f'Explained Var: {np.round(np.sum(pca_mod.explained_variance_ratio_), decimals=4)}', 
        transform=ax.transAxes)
ax.set_xlabel('PCA Dim 1')
ax.set_ylabel('PCA Dim 2')
ax.set_title('PCA Scatterplot for Encoder Features')
ax.legend(loc=4)
plt.savefig(f'{results_save_dir}/PCAencoding_pca_features.png')

NameError: name 'PCA' is not defined

#### Training Curves

In [29]:
dec_log, ptrain_log = pd.read_csv(f'{results_save_dir}/dec_log.csv'), pd.read_csv(f'{results_save_dir}/pretrain_log.csv')

In [30]:
fig, ax = plt.subplots(1, 3, figsize=(20, 5))
ptrain_log.plot(x='epoch', y='loss', ax=ax[0], title='Reconstruction Loss')
dec_log.plot(x='iter', y='acc', ax=ax[1], title='Clustering Accuracy')
dec_log.plot(x='iter', y='nmi', ax=ax[-1], title='Clustering NMI')
plt.savefig(f'{results_save_dir}/training_curves.png')

NameError: name 'plt' is not defined

## Ward-Link Agglomerative Clustering

### DEC Xception Training Regime

This part of the notebook defines the generator for the training regime of the DEC model over the features extracted from the Xception architecture

In [31]:
# Defining our DEC model
dec_model = DEC_Agglomerative([2048, 500, 500, 2000, 10], n_clusters=2)
dec_model.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 2048)              0         
_________________________________________________________________
encoder_0 (Dense)            (None, 500)               1024500   
_________________________________________________________________
encoder_1 (Dense)            (None, 500)               250500    
_________________________________________________________________
encoder_2 (Dense)            (None, 2000)              1002000   
_________________________________________________________________
encoder_3 (Dense)            (None, 10)                20010     
_________________________________________________________________
clustering (ClusteringLayer) (None, 2)                 20        
Total params: 2,297,030
Trainable params: 2,297,030
Non-trainable params: 0
_________________________________________________________________


In [32]:
results_save_dir = 'results/ward_link_agglomerative_clustering'
if not exist_directory(results_save_dir):
    os.makedirs(results_save_dir)

#### Pretraining

In [33]:
%%time
dec_model.pretrain(train_features, train_labels, epochs=100, save_dir=results_save_dir)

...Pretraining...
Epoch 1/100
59968/59968 [==============================] - 5s 86us/step - loss: 0.0153
        |==>  acc: 0.6569,  nmi: 0.0628  <==|
Epoch 2/100
59968/59968 [==============================] - 4s 66us/step - loss: 0.0080
Epoch 3/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0070
Epoch 4/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0067
Epoch 5/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0065
Epoch 6/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0064
Epoch 7/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0062
Epoch 8/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0062
Epoch 9/100
59968/59968 [==============================] - 4s 66us/step - loss: 0.0061
Epoch 10/100
59968/59968 [==============================] - 4s 65us/step - loss: 0.0060
Epoch 11/100
59968/59968 [==============================] - 4s 65

#### Clustering

In [34]:
%%time
dec_model.compile(optimizer=SGD(0.01, 0.9), loss='kld')
y_pred = dec_model.fit(train_features, y=train_labels, batch_size=32, save_dir=results_save_dir, 
                       cluster_linkage='complete')
print('acc:', metrics.acc(train_labels, y_pred))

Update interval 140
Save interval 9370
Initializing cluster centers with k-means.
Iter 0: acc = 0.55953, nmi = 0.00361, ari = -0.00314  ; loss= 0
saving model to: results/ward_link_agglomerative_clustering/DEC_model_0.h5
Iter 140: acc = 0.54229, nmi = 0.00183, ari = -0.00487  ; loss= 0.00211
Iter 280: acc = 0.52716, nmi = 0.00138, ari = -0.00434  ; loss= 0.00554
Iter 420: acc = 0.52031, nmi = 0.00089, ari = -0.00315  ; loss= 0.01552
Iter 560: acc = 0.52019, nmi = 0.00051, ari = -0.00233  ; loss= 0.04956
Iter 700: acc = 0.52021, nmi = 0.00027, ari = -0.00167  ; loss= 0.02963
Iter 840: acc = 0.51893, nmi = 0.00032, ari = -0.00175  ; loss= 0.03638
Iter 980: acc = 0.51614, nmi = 0.00028, ari = -0.00146  ; loss= 0.05082
Iter 1120: acc = 0.51928, nmi = 0.00020, ari = -0.00136  ; loss= 0.03777
Iter 1260: acc = 0.51974, nmi = 0.00008, ari = -0.00088  ; loss= 0.0519
Iter 1400: acc = 0.51963, nmi = 0.00001, ari = -0.00035  ; loss= 0.04321
Iter 1540: acc = 0.52149, nmi = 0.00002, ari = 0.00040  ;

### Visualization of the features extracted by the DEC model

#### Encoder Output Features

In [35]:
%%time
dec_encoder_model_pred = dec_model.encoder.predict(train_features)
pca_mod = PCA(2)
pca_mod.fit(dec_encoder_model_pred)
reduced_features = pca_mod.transform(dec_encoder_model_pred)

fig, ax = plt.subplots(1, 1)
sns.scatterplot(reduced_features[:, 0], reduced_features[:, 1], 
                hue=np.where(train_labels==0, 'elliptical', 'spiral'), ax=ax)
ax.text(0.02, 0.92, f'Explained Var: {np.round(np.sum(pca_mod.explained_variance_ratio_), decimals=4)}', 
        transform=ax.transAxes)
ax.set_xlabel('PCA Dim 1')
ax.set_ylabel('PCA Dim 2')
ax.set_title('PCA Scatterplot for Encoder Features')
ax.legend(loc=4)
plt.savefig(f'{results_save_dir}/PCAencoding_pca_features.png')

NameError: name 'PCA' is not defined

#### Training Curves

In [36]:
dec_log, ptrain_log = pd.read_csv(f'{results_save_dir}/dec_log.csv'), pd.read_csv(f'{results_save_dir}/pretrain_log.csv')

In [37]:
fig, ax = plt.subplots(1, 3, figsize=(20, 5))
ptrain_log.plot(x='epoch', y='loss', ax=ax[0], title='Reconstruction Loss')
dec_log.plot(x='iter', y='acc', ax=ax[1], title='Clustering Accuracy')
dec_log.plot(x='iter', y='nmi', ax=ax[-1], title='Clustering NMI')
plt.savefig(f'{results_save_dir}/training_curves.png')

NameError: name 'plt' is not defined